In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://pp

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-08 05:01:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.23MB/s    in 0.8s    

2021-12-08 05:01:53 (1.23 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1774101|R3PMQXEVYR4ZT1|B00DVKOYBM|     263046246|         8 Ball Pool|     Mobile_Apps|          3|            0|          0|   N|                Y|                 Fun|It is a fun game ...| 2015-08-31|
|         US|   13679234|R1I1K3EO98EMUV|B00R1IA4RS|     128868854|Christmas Cookie ...|     Mobile_Apps|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
from pyspark.sql.functions import col
 

In [6]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3PMQXEVYR4ZT1|          3|            0|          0|   N|                Y|
|R1I1K3EO98EMUV|          5|            0|          0|   N|                Y|
|R3K23URSI2MY42|          5|            4|          4|   N|                Y|
| RH2P770T10X38|          5|            0|          0|   N|                Y|
| RV9YBAW8CWFYN|          1|            0|          0|   N|                Y|
| R19627K7G1G6X|          5|            0|          0|   N|                Y|
|R22858XECEF7YQ|          3|            0|          0|   N|                N|
|R3KQOY7MDZOK0U|          1|            0|          0|   N|                Y|
|R31BKIN02QFDHS|          4|            0|          0|   N|                Y|
|R3FW62P5EUDUUV|          5|            0|          0|   N|     

In [7]:
#1. Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
#vine_20plus_votes= vine_df.filter("total_votes>=20")
vine_20plus_votes = vine_df.filter(vine_df['total_votes'] >= 20)
vine_20plus_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1Z9QZRQJ1F8JX|          1|           17|         22|   N|                Y|
|R31PAYBVBP8W62|          1|           54|         81|   N|                Y|
|R33GCMAT1K5DX8|          2|          141|        190|   N|                Y|
|R3MIL83RY4SMS4|          1|           27|         38|   N|                Y|
|R37T9FXGG07XOM|          1|           24|         28|   N|                Y|
|R2E9ZHX72173PZ|          1|           14|         21|   N|                Y|
|R2I6YA0BVG78P6|          3|           59|         70|   N|                Y|
|R1NB53BPRH24RK|          5|           15|         26|   N|                Y|
|R3AX1CCCZH20C5|          5|           17|         26|   N|                Y|
|R1FOM4K4S3S1A3|          4|           19|         22|   N|     

In [8]:
#2. Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
#helpful_votes_df = vine_20plus_votes.filter("helpful_votes/total_votes>=0.5")
helpful_votes_df = vine_20plus_votes.filter(vine_20plus_votes['helpful_votes']/vine_20plus_votes['total_votes']>=0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1Z9QZRQJ1F8JX|          1|           17|         22|   N|                Y|
|R31PAYBVBP8W62|          1|           54|         81|   N|                Y|
|R33GCMAT1K5DX8|          2|          141|        190|   N|                Y|
|R3MIL83RY4SMS4|          1|           27|         38|   N|                Y|
|R37T9FXGG07XOM|          1|           24|         28|   N|                Y|
|R2E9ZHX72173PZ|          1|           14|         21|   N|                Y|
|R2I6YA0BVG78P6|          3|           59|         70|   N|                Y|
|R1NB53BPRH24RK|          5|           15|         26|   N|                Y|
|R3AX1CCCZH20C5|          5|           17|         26|   N|                Y|
|R1FOM4K4S3S1A3|          4|           19|         22|   N|     

In [9]:
#3. Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
#paid_review_df = helpful_votes_df.filter("vine == 'Y'")
paid_review_df = helpful_votes_df.filter(helpful_votes_df['vine'] =='Y')
paid_review_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [11]:
#4. Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
unpaid_review_df= helpful_votes_df.filter("vine == 'N'")
unpaid_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1Z9QZRQJ1F8JX|          1|           17|         22|   N|                Y|
|R31PAYBVBP8W62|          1|           54|         81|   N|                Y|
|R33GCMAT1K5DX8|          2|          141|        190|   N|                Y|
|R3MIL83RY4SMS4|          1|           27|         38|   N|                Y|
|R37T9FXGG07XOM|          1|           24|         28|   N|                Y|
|R2E9ZHX72173PZ|          1|           14|         21|   N|                Y|
|R2I6YA0BVG78P6|          3|           59|         70|   N|                Y|
|R1NB53BPRH24RK|          5|           15|         26|   N|                Y|
|R3AX1CCCZH20C5|          5|           17|         26|   N|                Y|
|R1FOM4K4S3S1A3|          4|           19|         22|   N|     

In [ ]:
#5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

In [33]:
# total count of reviews
unpaid_reviews = unpaid_review_df.count()
paid_reviews = paid_review_df.count()

# the number of 5-star reviews,
unpaid_5star_reviews = unpaid_review_df.filter("star_rating == 5").count()
paid_5star_reviews = paid_review_df.filter("star_rating == 5").count()

# percentage of 5-star reviews for the two types of review (paid vs unpaid).
unpaid_5star_review_percentage= (unpaid_5star_reviews/unpaid_review_df.count())*100
#since there are no paid review we cannont calculate paid 5 star review percentage 
paid_5star_review_percentage = 0 #(paid_5star_reviews/paid_review_df.count())*100

print(f'Unpaid reviews are: {unpaid_reviews}')
print(f'Paid reviews are: {paid_reviews}')
 
print(f'Unpaid 5-star reviews are: {unpaid_5star_reviews}')
print(f'Paid 5-star reviews are: {paid_5star_reviews}')

print(f'Unpaid percentage of 5-star reviews are: {unpaid_5star_review_percentage}%')
print(f'Paid percentage of 5-star reviews are: Not available')

Unpaid reviews are: 129516
Paid reviews are: 0
Unpaid 5-star reviews are: 59278
Paid 5-star reviews are: 0
Unpaid percentage of 5-star reviews are: 45.76886253435869%
Paid percentage of 5-star reviews are: Not available


In [ ]:
 unpaid_review_df.describe().show() 

+-------+--------------+------------------+------------------+-----------------+------+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|      total_votes|  vine|verified_purchase|
+-------+--------------+------------------+------------------+-----------------+------+-----------------+
|  count|        129516|            129516|            129516|           129516|129516|           129516|
|   mean|          null|3.4708298588591373| 72.90205071188116|88.69631551314123|  null|             null|
| stddev|          null|1.6906143877658344|191.76724629371475|223.3263140151915|  null|             null|
|    min|R1000Q8CAQ6DF4|                 1|                10|               20|     N|                N|
|    max| RZZY42OKOMFKO|                 5|             10466|            12016|     N|                Y|
+-------+--------------+------------------+------------------+-----------------+------+-----------------+



In [34]:
 paid_review_df.describe().show() 

+-------+---------+-----------+-------------+-----------+----+-----------------+
|summary|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-------+---------+-----------+-------------+-----------+----+-----------------+
|  count|        0|          0|            0|          0|   0|                0|
|   mean|     null|       null|         null|       null|null|             null|
| stddev|     null|       null|         null|       null|null|             null|
|    min|     null|       null|         null|       null|null|             null|
|    max|     null|       null|         null|       null|null|             null|
+-------+---------+-----------+-------------+-----------+----+-----------------+

